In [5]:
from datasets import load_dataset


load_dataset("/vol/bitbucket/jg2619/augmenting_llms/benchmarks/ASDiv/", split="train")

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 97.86it/s]


Dataset text downloaded and prepared to /homes/jg2619/.cache/huggingface/datasets/text/ASDiv-a373d3734b174a1b/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


Dataset({
    features: ['text'],
    num_rows: 238
})

In [14]:

from bs4 import BeautifulSoup
 
 
# Reading the data inside the xml
# file to a variable under the name
# data
with open('ASDiv.xml', 'r') as f:
    data = f.read()
 
# Passing the stored data inside
# the beautifulsoup parser, storing
# the returned object
Bs_data = BeautifulSoup(data, "lxml")


# Finding all instances of tag
# `unique`
b_unique = Bs_data.find_all('problem')
 
#print(b_unique)
 
# Using find() to extract attributes
# of the first instance of the tag
b_name = Bs_data.find('problem', {'id':'nluds-0001'})
 
print(b_name)
 
# Extracting the data stored in a
# specific attribute of the
# `child` tag
value = b_name.get('test')
 
print(value)

/vol/bitbucket/jg2619/toolformer-luci/oldtoolvenv/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


<problem grade="1" id="nluds-0001" source="http://www.k5learning.com">
		Seven red apples and two green apples are in the basket.
		<question>How many apples are in the basket?</question>
<solution-type>Addition</solution-type>
<answer>9 (apples)</answer>
<formula>7+2=9</formula>
</problem>
None


In [71]:
problem_ids = []
# Read problem ids from /vol/bitbucket/jg2619/augmenting_llms/benchmarks/ASDiv/fold0.txt file, which has lines such as:
# nluds-0153
# nluds-2291
# etc

import re

with open('/vol/bitbucket/jg2619/augmenting_llms/benchmarks/ASDiv/fold0.txt', 'r') as f:
    for line in f.readlines():
        problem_ids.append(line.strip())

questions = []
answers = []

for id in problem_ids:
    problem = Bs_data.find("problem", id=id)
    questions.append(str(problem.find(text=True, recursive=False)[3:-3] + " " + problem.question.text))
    answers.append(re.sub('[^0-9.]', '', problem.answer.text))

/tmp/ipykernel_1351433/4071319210.py:19: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  questions.append(str(problem.find(text=True, recursive=False)[3:-3] + " " + problem.question.text))


In [70]:
# Strips string of all non-numeric or fullstop characters:
def strip_non_numeric(string):
    return re.sub('[^0-9.]', '', string)


In [72]:
answers

['10',
 '76',
 '3',
 '3',
 '9',
 '65',
 '10',
 '2',
 '12',
 '28',
 '7',
 '45',
 '9',
 '3',
 '60',
 '6',
 '6',
 '77',
 '4',
 '155',
 '20',
 '48',
 '7',
 '6',
 '5',
 '8',
 '9',
 '5',
 '8',
 '23',
 '16',
 '19',
 '45',
 '127',
 '34',
 '43',
 '16',
 '275',
 '39',
 '18',
 '9',
 '9',
 '15',
 '10',
 '16',
 '11760',
 '6',
 '83',
 '14',
 '93',
 '11',
 '13',
 '8',
 '98',
 '18',
 '87',
 '11',
 '44',
 '19',
 '30',
 '45',
 '7',
 '8',
 '10',
 '6',
 '12',
 '5',
 '100',
 '3',
 '39',
 '38',
 '9',
 '18',
 '44',
 '11',
 '60',
 '7',
 '42',
 '18',
 '94',
 '28',
 '9',
 '22',
 '17',
 '38',
 '7',
 '929',
 '11',
 '62',
 '6',
 '46',
 '5',
 '55',
 '55',
 '12',
 '17',
 '37',
 '15',
 '61',
 '40',
 '149',
 '18',
 '85',
 '78',
 '40',
 '5',
 '54',
 '240',
 '7',
 '120',
 '30',
 '180',
 '2',
 '164',
 '39',
 '109',
 '3',
 '82',
 '64',
 '98',
 '6',
 '385',
 '12',
 '25',
 '22',
 '48',
 '5',
 '45',
 '4',
 '182',
 '35',
 '174',
 '117',
 '22',
 '4',
 '12',
 '24',
 '12',
 '2',
 '9',
 '7',
 '6',
 '3',
 '2',
 '7',
 '8',
 '12',
 

In [13]:
# Import GPTJ tokenizer:

import torch

cache_dir = "/vol/bitbucket/jg2619/augmenting_llms/augmented_data_pipeline/toolformer/cache"

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B", cache_dir=cache_dir)


tokenizer.add_tokens(["[PAD]"])
tokenizer.pad_token="[PAD]"


print(tokenizer.decode([   50, 14715,   290, 21946,   389,  1266,  2460,  2712,   319, 12330,
         9669,  3466,    13,   383,  3466,   423,   734,  3357,  1830,  7530,
           13,   554,   262,   717,   983,    11, 32151,   550,   262,  1363,
         2184,  4621,   290,  7781,  1478,  2173,    13, 21946,  7781,   604,
         7380,  2173,   621, 32151,   287,   262,   976,   983,    13, 32151,
         1464, 12766,  1141,  1497,  1830,   290,   511,  1218,  2872,   373,
          379, 21946,   338,  1363,  2184,    13, 32151,  7781,   718,  7380,
         2173,   287,   262,  1218,   983,   621, 21946,   338,  4776,   287,
          262,   717,   983,    13,  1374,   867,  2472,  2173,   750, 32151,
         4776,  1141,  1111,  1830,    30]))
print()

print(torch.tensor([15309,    11,  7299,    11,   290,  1215,   417, 18829,    68,   389,
         2111,   284,  4929,   606,   477,    11, 14878,   326,   318,    13,
          220, 17083,   484,   423,  4978, 42489, 14878,    13,   220,  4422,
          468,  4978,   642,   517,   621,  7299,    11,   290,  7299,   468,
         4978,  1511,  1342,   621,   604,  1661,   355,   867,   355,  1215,
          417, 18829,    68,   468,  4978,    13,  1374,   867, 14878,   468,
         7299,  4978,    30]).max().item())

print(tokenizer.decode([  198,   198,  8600,   416,  2239, 14607,    25,   383,   717,   983,
          373,   379, 32151,   338,  1363,  2184,    11,   290,   262,  1218,
          983,   373,   379, 21946,   338,  1363,  2184,    13,  8447,    11,
        32151,   338,  4776,   287,   262,   717,   983,   373,   685,  9771,
         3129,  1352,     7]))

print()

print(tokenizer.decode([ 4821,   284,   663, 24000,  7508,    11,   257,  6131,   286, 12014,
          468,  8646, 14653,   583,  7351,    13,  1002,   257,  5867,    70,
         6131,   468,   642, 43096,    11,   703,   867, 16379,   460,   345,
         4483,   611,   534,  4445, 28740,  2496,   318,  4751,   290,   345,
          423,  1541, 13529, 21431, 14653,    30]))

print(tokenizer.decode([  198,   198,  8600,   416,  2239, 14607,    25,   383,   717,   983,
          373,   379, 32151,   338,  1363,  2184,    11,   290,   262,  1218,
          983,   373,   379, 21946,   338,  1363,  2184,    13,  8447,    11,
        32151,   338,  4776,   287,   262,   717,   983,   373,   685,  9771,
         3129,  1352,     7],[  198,   198,  8600,   416,  2239, 14607,    25,   317,  6131,   286,
        12014,   468,  8646, 14653,   583,  7351,    13,  8447,    11,   611,
          345,   460,  4483,   642, 11668,    11,   340,   468,   685,  9771,
         3129,  1352,     7]))

print()


print(tokenizer.decode([ 2953, 31908,   338,  2156,    11,   612,   318,  5403,   355,   881,
        11676,   355,  6463,  8506,    13,   679,   468,   257,  2472,   286,
         2319,  6463,  8506,   287,   465,  2156,    13, 31908,  5839,  3126,
          517,  6463,  8506,   379,   262,  3650,   290,  2319,  7380,  1162,
         5907,   621,   262,  1271,   286,  6463,  8506,    13,  9938,   262,
         5929,  2472,   286,   262,  1271,   286,  1162,  5907,   290,  6463,
         8506, 31908,   468,   287,   262,  2156,    30]))

print(tokenizer.decode([  198,   198,  8600,   416,  2239, 14607,    25,   383,   717,   983,
          373,   379, 32151,   338,  1363,  2184,    11,   290,   262,  1218,
          983,   373,   379, 21946,   338,  1363,  2184,    13,  8447,    11,
        32151,   338,  4776,   287,   262,   717,   983,   373,   685,  9771,
         3129,  1352,     7],[  198,   198,  8600,   416,  2239, 14607,    25,   317,  6131,   286,
        12014,   468,  8646, 14653,   583,  7351,    13,  8447,    11,   611,
          345,   460,  4483,   642, 11668,    11,   340,   468,   685,  9771,
         3129,  1352,     7],[  198,   198,  8600,   416,  2239, 14607,    25, 31908,   468,   685,
         9771,  3129,  1352,     7]))

print()

print(tokenizer.decode([  44, 3093,   77,  338,  717, 1700, 2702,  838, 1661,  355,  867, 9088,
         355, 2113, 1940,  338,   13, 1002,  484, 2702, 9193,   11,  830, 9088,
        5929,   11,  703,  867, 9088,  750, 2113, 1940, 3677,   30]))

print(tokenizer.decode([  198,   198,  8600,   416,  2239, 14607,    25,   383,   717,   983,
          373,   379, 32151,   338,  1363,  2184,    11,   290,   262,  1218,
          983,   373,   379, 21946,   338,  1363,  2184,    13,  8447,    11,
        32151,   338,  4776,   287,   262,   717,   983,   373,   685,  9771,
         3129,  1352,     7],[  198,   198,  8600,   416,  2239, 14607,    25,   317,  6131,   286,
        12014,   468,  8646, 14653,   583,  7351,    13,  8447,    11,   611,
          345,   460,  4483,   642, 11668,    11,   340,   468,   685,  9771,
         3129,  1352,     7],[  198,   198,  8600,   416,  2239, 14607,    25, 31908,   468,   685,
         9771,  3129,  1352,     7],[  198,   198,  8600,   416,  2239, 14607,    25,   685,  9771,  3129,
         1352,     7]))


print(tokenizer.decode([  198,   198,  8600,   416,  2239, 14607,    25,   383,   717,   983,
          373,   379, 32151,   338,  1363,  2184,    11,   290,   262,  1218,
          983,   373,   379, 21946,   338,  1363,  2184,    13,  8447,    11,
        32151,   338,  4776,   287,   262,   717,   983,   373,   685,  9771,
         3129,  1352,     7],[  198,   198,  8600,   416,  2239, 14607,    25,   317,  6131,   286,
        12014,   468,  8646, 14653,   583,  7351,    13,  8447,    11,   611,
          345,   460,  4483,   642, 11668,    11,   340,   468,   685,  9771,
         3129,  1352,     7],[  198,   198,  8600,   416,  2239, 14607,    25, 31908,   468,   685,
         9771,  3129,  1352,     7],[  198,   198,  8600,   416,  2239, 14607,    25,   685,  9771,  3129,
         1352,     7],[  198,   198,  8600,   416,  2239, 14607,    25,   383,  1271,   286,
        14878,   326,  4422,   468,  4978,   318,   685,  9771,  3129,  1352,
            7]))



Sasha and Julie are best friends playing on opposing basketball teams. The teams have two practice games scheduled. In the first game, Sasha had the home court advantage and scored 14 points. Julie scored 4 fewer points than Sasha in the same game. Sasha always struggles during away games and their second match was at Julie's home court. Sasha scored 6 fewer points in the second game than Julie's score in the first game. How many total points did Sasha score during both games?

42489


Step by step reasoning: The first game was at Sasha's home court, and the second game was at Julie's home court. Therefore, Sasha's score in the first game was [Calculator(

According to its nutritional info, a bag of chips has 250 calories per serving. If a 300g bag has 5 servings, how many grams can you eat if your daily calorie target is 2000 and you have already consumed 1800 calories?


TypeError: argument 'skip_special_tokens': 'list' object cannot be converted to 'PyBool'